# Question:
Develop  a  predictive  model  in  which  the  dependent variable  isCvdVax_DisparityY.  Your  task  is  to examine  the  role  of  different  socioeconomic and  demographic  variables  indetermining  thecovid-19 vaccination rate disparities. Note following points to complete this project. 

(1)You  are  free  to  choose  any prediction/regression model  to  examine  the  role  of  predictors  in determining CvdVax_DisparityY.  But  use  only  data labelledas “train” for the model building purpose. Evaluate the performance ofyourmodels on the test data.

(2)Report three models that give best result on the test dataset. Mentionthe final model that you would select. Explainthe reason for its selection. 

(3)How would you assess the importance of different predictors in your model? Which predictors are most important in determining the racial disparity in covid-19 vaccinate rate? 

(4)Areyour  findings  similar  to  the resultsreported  by  authors  in  Table  1of  the  article  (see  the hyperlink above)? If not, why? 

(5)Upload Python/R code with your project report. 

# IME-project
- Examine the relationship between social determinants of health and racial disparitiesin covid-19vaccinationat the county level in US
- dates worked:
    - date: 28-09 --> understanding the data variables

In [1]:
import numpy as np
import pandas as pd

In [49]:
data=pd.read_csv('ime692_project.csv')

# seperate data into test and train based on `Test` indicator
pd.options.mode.chained_assignment = None  # default='warn' --> to supress chain assignment warning <--
test_data=data.loc[data.Test==1] ## previous indices could be useful ##
train_data=data.loc[data.Test==0]
print("==========================================")
print("No of testing observations: ",test_data.shape[0])
print("No of training observations: ",train_data.shape[0])
print("==========================================")

# drop the indicator inplace
test_data.drop('Test',axis=1,inplace=True)
train_data.drop('Test',axis=1,inplace=True)

# sepearate predicted and regressor variables
y_test_data=test_data['CvdVax_DisparityY'].tolist()
X_test_data=test_data.drop('CvdVax_DisparityY',axis=1)
y_train_data=train_data['CvdVax_DisparityY'].tolist()
X_train_data=train_data.drop('CvdVax_DisparityY',axis=1)


No of testing observations:  225
No of training observations:  531


In [51]:
X_train_data

,State,County,IT_WholeRate,HighSchool_WholeRate,MedianInc_WholeAvg,republican_rate,Segregation,urban,racial_weighted_bias,hesitancy,HighSchool_Disparity,IT_Disparity,MedianInc_Disparity,vehicle,FacNumRate,CaseRate,Black_Prop
0,California,Alameda,91.4,88.4,99.406,1770.559475,49.890444,1,33.258827,9,5.7,10.7,62.678,90.4,0.017292,5.114553,11.031221
1,California,Butte,89.1,89.2,52.537,4775.484604,62.099507,1,36.520664,13,0.7,6.4,24.189,92.8,0.017793,5.288203,1.897019
2,California,Colusa,86.5,71.3,59.401,5726.667504,19.219010,0,37.223305,14,18.7,-0.6,39.137,94.6,0.018564,10.205597,1.308767
4,California,Del Norte,84.0,80.1,45.283,5641.809291,35.769303,0,36.879187,15,21.5,-12.6,-57.687,90.4,0.028765,4.631094,3.523659
5,California,El Dorado,90.1,93.4,83.377,5324.527717,58.879477,1,36.525529,10,8.4,3.6,-4.821,95.9,0.017112,5.069927,1.037113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Wisconsin,Douglas,83.3,93.7,53.986,4439.702475,73.099649,1,36.775647,17,15.1,-6.2,-0.735,92.7,0.041715,10.801854,1.253766
752,Wisconsin,Dodge,87.6,90.8,60.652,6476.031146,65.224838,0,35.934490,17,12.3,0.2,26.775,94.9,0.013661,14.317103,3.345894
753,Wisconsin,Dane,91.4,95.8,73.893,2293.396669,49.608129,1,35.577848,14,8.4,9.0,42.958,92.7,0.016645,8.236402,5.531969
754,Wisconsin,Brown,89.8,92.1,62.340,5282.318703,58.983900,1,37.440183,17,12.5,6.1,40.201,94.5,0.018145,13.017971,2.976465


In [6]:
## initial data analysis
print(data.shape)
print(data.columns)

(756, 19)
Index(['State', 'County', 'IT_WholeRate', 'HighSchool_WholeRate',
       'MedianInc_WholeAvg', 'republican_rate', 'Segregation', 'urban',
       'racial_weighted_bias', 'hesitancy', 'HighSchool_Disparity',
       'IT_Disparity', 'MedianInc_Disparity', 'CvdVax_DisparityY', 'vehicle',
       'FacNumRate', 'CaseRate', 'Black_Prop', 'Test'],
      dtype='object')


dtype('O')

### Which variables to use(PCA):
- All variables are numeric except State and County Name -
    - also urban(1)/rural(0): but can I keep it off for the moment? IG yes.
- Normalization: which `z-score` or `min-max`
- Role of predictors: `some score based approach ?`
    - disparity variables must be included for `y`
    - Other factors also affect but are they worth including with model complexity there
    - Variable matching(6): `overall -- difference`
        - IT_WholeRate -- IT_Disparity
        - MedianInc_WholeAvg -- MedianInc_Disparity
        - HighSchool_WholeRate -- HighSchool_Disparity
    - `Seems` Imp for `y`: 
        - racial_weighted_bias
        - Segregation
        - republican_rate
        - `hesitancy` :smoking: ?
        - FacNumRate(economic)
    - `Less` imp for `y`:
        - vehicle(economic)
        - CaseRate
        - Black_Prop
- To be honest all seems important; that is why they included right ... but relatievly which ones are more and less ... that need to be seen
- real question to answer would be is there any role of previliages(economic-social) and political alliances; it seems so from intuitoin but what does the data tell us?

### Which models for prediction/regression of `y`? 
1. A: basic things is to keep model as simple as possible with few variables to exaplain(`PCA` can be done here to check explained variability by a particular variable/regressor), if possible.
    - Multi-linear regression model (linear or not?): covariance or other measures of linearity could be used to check.
    - Decision tree: regression trees
    - NN 😄
    - Prob. gaussian(linear special case) model

### Concepts from class:
- PCA: to check variability explained by variables and dim reduction
- Linearity measures{corr, factor analysis -- see below}
- Sampling: bootstrap and cross-validation
- Basic pre-model concepts: z-score, min-max normalizations
- Models: 
    - high dim regressions: 
        - lasso 
        - lar
    - logistics regression? Nah, it is for categorical(usually binary) predictive variable `mainly`
    - Ensemble Models: Ensemble models use multiple machine learning algorithms to obtain better predictive performance than what could be obtained from one algorithm alone.
    - Factor Analysis: Factor analysis is a method used to describe variability and aims to find independent latent variables.
    - Random forsts: ?? -- > ensembling of multiple regression trees
    - Generalized Linear Model (GLM) for Two Values ??

## Pre-model steps
- Normalization of numeric variables
    - why? 